In [3]:
import numpy as np

# PCA for Linear Algebra Project

Principal component analysis, or PCA, is a dimensionality reduction method that is often used to reduce the dimensionality of large data sets, by transforming a large set of variables into a smaller one that still contains most of the information in the large set.


### Centering data:
We need to subtract the mean of each variable from all observations (so that the data has a mean of 0). This is important because PCA is sensitive to the scale of the variables.

For each vector $x_i$ of the data:

$$x_i^{\prime}​=x_i​ -  \overline{x_i}​​$$

where:
- $\overline{x_i} = \frac{\sum_{i=0}^n{x_i}}{n}$

Now combine the data into one matrix:

$$X_{cent} = 
\begin{pmatrix}
-x_1-\\
-x_2-\\
...\\
-x_n-
\end{pmatrix}$$
### Finding covariance matrix ($C$):

PCA (principal component analysis) looks for the directions (components) along which the data has the most variance. To find these directions, we need to understand how the variables "variate" together — that is, how one feature varies relative to another. So we need covariance matrix, which is a square matrix of size $n×n$, where $n$ is the number of features. Each element $cov(x_i,x_j)$ in this matrix shows how much two features $x_i$ and $x_j$ change together.

Calculate the matrix:
$$C = \frac{1}{n-1}X_{cent}^TX_{cent}$$
where:
- $X_{cent}$ $-$ Matrix of data

### Eigenvalues and eigenvectors of $C$

The eigenvectors correspond to the directions in which the data have the greatest variance, and the eigenvalues determine the "strength" (importance) of each of these directions.
Find the eigenvalues:
$$det(C-\lambda I) = 0$$
$$(C-\lambda I)v = 0$$
where: 
- $\lambda$ $-$ eigenvalue
- $v$ $-$ eigenvector

But for computing used <a href="https://en.wikipedia.org/wiki/Power_iteration">Power Method</a> with Rayleigh quotient. So formulas will be:

The power iteration algorithm starts with a vector $b_0$, which may be an approximation to the dominant eigenvector or a random vector. The method is described by the recurrence relation

$$b_{k+1} = \frac{Ab_k}{‖Ab_k‖}$$

So, at every iteration, the vector $b_{k}$ is multiplied by the matrix $A$ and normalized. 

Eigenvalue:

$$ \mu_k = \frac{b^*_kAb_k}{b^*_kb_k}$$

converges to the dominant eigenvalue (with <a href="https://en.wikipedia.org/wiki/Rayleigh_quotient">Rayleigh quotient</a>).

**Selecting principal components:**

The eigenvectors corresponding to the largest eigenvalues become the principal components. The number of principal components is usually determined based on how much of the total variance you want to retain($n$).
So, the transformation matrix will be $V_k$ (matrix of the eigenvectors of the $n$ principal components).

### Transforming data

The final step is to project the data into the principal component space. This is done by multiplying the data matrix ($X$) by the eigenvector matrix ($V_k$).
$$X_{pca} = X_{cent}V_k$$

<mark>Additional notes:</mark>
- Calculating norm is by $\sqrt{\sum^n_{i=1}{x_i^2}}$ where $x_i - $ feature.
- Eigenvalues are sorted and eigenvectors corresponding them.
- Taking components by $\frac{\sum_{i=1}^k\lambda_i}{\sum_{i=1}^n\lambda_i}\le 0.99$ (function **<span style = "color: lightblue">def</span> <span style = "color: lightgreen">_choose_components</span>**).

In [ ]:
class PowerPCA:

    def _powerIter(self, A):
        n = A.shape[1]
        v_k = np.random.rand(n)
        while True:
            v_k1 = np.dot(A, v_k)
            v_k1_norm = self._norm(v_k1.T)
            v_k1 = v_k1 / v_k1_norm

            if v_k1_norm < 1e-10:
                return 0.0, np.zeros(n)

            if self._norm(np.abs(v_k1 - v_k)) < 1e-10:
                v_k = v_k1
                break
            v_k = v_k1
        eigval = np.dot(np.conj(v_k).T, np.dot(A, v_k)) / np.dot(np.conj(v_k).T, v_k)
        v = v_k
        return eigval, v_k1

    def _eig(self, A, cout: bool = False):
        eigenvalues = []
        eigenvectors = []
        A_copy = A.copy()
        n = A_copy.shape[1]

        for i in range(n):
            eigval, v = self._powerIter(A_copy)

            if self._norm(v) < 1e-8 or np.isnan(eigval):
                continue

            eigenvalues.append(eigval)

            v = v / self._norm(v)
            for v_prime in eigenvectors:
                v -= np.dot(v_prime, v) * v_prime
            eigenvectors.append(v)
            A_copy = A_copy - eigval * np.outer(v, v)

        while len(eigenvalues) < n:
            eigenvalues.append(0.0)
            eigenvectors.append(np.zeros(n))
        
        if cout:
            print(f"EigResult(eigenvalues={np.array(eigenvalues)}), eigenvectors={np.array(eigenvectors)}))")
        return np.array(eigenvalues), np.array(eigenvectors).T
    
    def _norm(self, v):
        sum_sq = 0.0
        for i in range(len(v)):
            sum_sq += v[i] ** 2
        return sum_sq ** 0.5
    
    def _choose_components(self, eigenvalues, threshold=0.99):
        total = sum(eigenvalues)
        running_sum = 0.0
        for i, eigval in enumerate(eigenvalues):
            running_sum += eigval
            if running_sum / total >= threshold:
                return i + 1
        return len(eigenvalues)
    
    
    def _centerData(self, X, cout = False):
        mu = np.mean(X, axis=0)
        X_cent = X - mu
        return X_cent
    
    def fit_transform(self, data):
        centeredData = self._centerData(data)
    
        C = (1 / (centeredData.shape[0] - 1)) * centeredData.T @ centeredData
        eigvals, eigvecs = self._eig(C)
    
        idx = np.argsort(eigvals)[::-1]
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
    
        k = self._choose_components(eigvals)
        eigvecs = eigvecs[:, :k]
    
        return centeredData @ eigvecs


### All links together:

- <a href="https://en.wikipedia.org/wiki/Power_iteration">Power Method</a>
- <a href="https://en.wikipedia.org/wiki/Rayleigh_quotient">Rayleigh quotient</a>